# Plotting With Gadfly in ≤ 15 Minutes
Jason A. Grafft

jason@grafft.co

1. Introduction
2. `@>> ["Julia", "Gadfly"] map(x -> "Why $x?")`
3. DataFrame-type structures
4. Plotting with Gadfly
5. Resources

## 1. Introduction

In [ ]:
# Library for lazy loading
using Lazy

# Libraries for data handling
using CSV, DataFrames, Query

# Library-in-question
using Gadfly

# Load some helper functions for generating data
plus001(n::Float64) = @lazy n+0.001:plus001(n+0.001);
epoch_ll = @> 1530426514. plus001;

df(n::Int64) = DataFrame(t=[x for x in take(n, epoch_ll)], x=rand(n), y=rand(n), z=rand(n));
df_ll(n::Int64) = @lazy df(n):df_ll(n);

sensorXYZ(df::DataFrame) = @>> [(:x, colorant"deepskyblue"), (:y, colorant"orange"), (:z, colorant"green")] map(s -> plot(df, x=:t, y=s[1], color=[s[2]], Guide.xticks(label=false), Guide.ylabel("$(s[1])"), Geom.smooth)) hstack

_layer(t) = layer(x=:t, y=t[1], color=[t[2]], Geom.smooth)

themedPlot(df::DataFrame) = plot(df, _layer((:x, colorant"deepskyblue")), _layer((:y, colorant"orange")), _layer((:z, colorant"green")), Guide.ylabel(""), Guide.manual_color_key("Column", ["x", "y", "z"], ["deepskyblue", "orange", "green"]))

## 2. `@>> ["Julia", "Gadfly"] map(x -> "Why $x?")`
1. Julia
    - Python/R inspired syntax
    - Exceptionally performant
    - Purpose-built for numerical computing
    - Friendly to algebraic structures, functional programming
2. Gadfly
    - Built on Julia
    - SVG drawing package
    - Friendly to \*\*flow programming patterns

## 3. DataFrame-type structures
DataFrame-type structures are familiar. They are ignorantly opinionated, and we love them for it.

In [ ]:
@> 12 df

Relationships in the physical world are not strictly 2-dimensional, however. "In the wild", the DataFrames in each list contained by `sensorData` must be aggreated before they can be compared with another set (list) of DataFrames. Formally, we have

$$\{\ell, L\} \in List$$
$$\forall\ell\in L: \ell\ni DataFrame$$

Pragmatically, we use nesting to partially encode our data. This provides critical context (state) to the functions tasked to process it.

In our current case, each set of sensors, $\ell$, provides a master control system with a partial view of the surrounding environment. Compressing $\ell$ into a single value (object) prior to reporting upward allows the control system to more efficiently reason.

In [ ]:
# Lazy list of "sensor" data where each inner list is of length 5 and each DataFrame has 15,345 entries--
# ~15 seconds' worth of samples at 1000hz.
sensorArray = @lazy take(5, df_ll(15345)):sensorArray;

# Lazy list of six (6) lists of length five (5).
rawSensorData = @>> sensorArray take(6);

## 4. Plotting with Gadfly
- Standard plots readily available [[Gadfly.Geometries](http://gadflyjl.org/stable/lib/geometries.html)]
- Programmatic composition allows for arbitrary interleaving of logic
    - No need to coerce data into fixed structures or relations
    - Control of mutation, edge cases, missing values, ... in data
    - Data structuring schemas are highly visible (open source)
- Plots are values (objects)
    - Custom titles/axes/file names/...
    - Multiple computations/settings/data sets/...
    - "Low overhead" modifications
    - Layering and stacking
- Draws to SVG, SVGJS
    - PNG, PDF, and PS via Cairo backend
- Jupyter and REPL

In [ ]:
# Plot with largely default values
sen35 = head(rawSensorData[3][5],5000)
plot(sen35,
        x=:t,
        y=Col.value(:x, :y, :z),
        color=Col.index(:x, :y, :z),
        Guide.ylabel(""),
        Geom.smooth
    )

In [ ]:
# Data above with (x, y, z) plotted separately
@>> [:x, :y, :z] map(s -> plot(sen35, x=:t, y=s, Guide.xticks(label=false), Guide.ylabel(""), Geom.smooth)) hstack

In [ ]:
# Add some color
@> sensorXYZ(sen35)

In [ ]:
themedPlot(sen35)

In [ ]:
# View plots in a grid
sen2 = @>> rawSensorData[2] map(x -> @> x head(5000)) map(x -> @> x sensorXYZ);
vstack(sen2[1], sen2[2], sen2[3])

We want to apply this color scheme to each DataFrame in our List. We will use a function which maps over $L$ and converts each `DataFrame` in each $\ell$ to a `Gadfly.Plot` object. Formally,

$$f: L \rightarrow P$$ where
$$\{\ell, P\} \in List$$
$$\forall\ell\in P: \ell\ni Gadfly.Plot$$

In [ ]:
P = @>> rawSensorData map(x -> @>> x map(y -> @> y head(5000) themedPlot));

In [ ]:
@> [colorScheme(sen35), P[3][5]] hstack

In [ ]:
# Remove (x, y) labels
# Highlight line segments < 0.495

## 5. Resources

* [Julia][jl]
    * [Julia Computing][jlc]
    * [JuliaBox][jbox]
* [Gadfly][gadfly]
* [DataFrames][df]
* [Jupyter][jup]

[df]: http://juliadata.github.io/DataFrames.jl/stable/
[gadfly]: http://gadflyjl.org/stable/
[jbox]: http://juliabox.com/
[jl]: https://julialang.org/
[jlc]: https://juliacomputing.com/
[jup]: http://jupyter.org/